In [1]:
import os
import sys
import numpy as np
from PIL import Image
import csv
import torchvision.transforms as transforms

c:\Users\Goncalo\miniconda3\envs\APA2022\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
list_path = "YOLO/data/KITTI/dataset.txt"

with open(list_path, "r") as file:
    img_files = file.readlines()

label_files = [path.replace("Imgs",os.path.join('labels')).replace('.png','.txt') for path in img_files]

k = 0
for j in range(0,len(img_files)-1):
    img_path = img_files[j % len(img_files)].rstrip()

    # Extract image as PyTorch tensor
    img = transforms.ToTensor()(Image.open(img_path))

    # Handle images with less than three channels
    if len(img.shape) != 3:
        img = img.unsqueeze(0)
        img = img.expand((3, img.shape[1:]))

    _, h, w = img.shape
    h_factor, w_factor = (h, w)

    # label
    label_path = label_files[j % len(img_files)].rstrip()

    if os.path.exists(label_path):
            boxes = np.loadtxt(label_path).reshape(-1, 5)

            # Extract coordinates
            x1 = w*(boxes[:, 1] - boxes[:, 3] / 2)
            y1 = h*(boxes[:, 2] - boxes[:, 4] / 2)
            x2 = w*(boxes[:, 1] + boxes[:, 3] / 2)
            y2 = h*(boxes[:, 2] + boxes[:, 4] / 2)
            
            # Convert tensor to PIL Image
            img = transforms.ToPILImage()(img)
            
            # Crop Image
            for i in range(len(boxes)):    
                cropped_img = img.crop((x1[i],y1[i],x2[i],y2[i]))
                cropped_img = cropped_img.resize((128, 128))
                #cropped_img.show()
                cropped_img.save("D:\APA\APA-Work2\CropDataset\id{0}_{1}.png".format(boxes[i,0],k))
                k+=1

